## **Compute risk bound and theta with Selection with Guaranteed Risk (SGR)**

Reference: https://arxiv.org/pdf/1705.08500 Algorithm 1

In [1]:
from risk_control import *
import numpy as np
import torch

## **Get Confidence Scores**

#### Method1 : **Maxprob**

In [2]:
from collections import Counter

def max_occurence(labels):
    count = Counter(labels)
    max_num = max(count, key=count.get)
    return max_num


    

In [3]:
data_path = "/teamspace/studios/this_studio/Selective_Prediction_VQA/predictions/logits_and_labels/"
NUM_BATCH = 2139
residuals = []
kappa = []
softmax = torch.nn.Softmax(dim=1)
for batch_no in range(NUM_BATCH):
    file_name = "Logits_and_labels" + str(batch_no) + ".pt"
    data = torch.load(data_path + file_name)
    
    for logits, labels in zip(data['logits'], data['labels']):
        if(len(labels)) == 0:
            continue
        
        logits = torch.from_numpy(logits)
        prob = softmax(logits)
        # print(prob)
        # print(torch.max(prob).numpy())
        kappa.append(torch.max(prob).numpy())
        idx = torch.argmax(prob)
        # print(prob[0][idx])
        # print(labels)
        residuals.append(idx.item() != max_occurence(labels))
        # print(kappa)
        # print(residuals)
        # break
    # break
        
    

In [4]:
kappa = np.array(kappa)
residuals = np.array(residuals)
risk_dict_max_prob = {}
risk_stars = [0.02, 0.1, 0.15, 0.20, 0.25,0.30, 1.0] # desired risk
delta = 0.01 ## confidence
for desired_risk in risk_stars:
    bound_cal = risk_control()
    [theta, b_star] = bound_cal.bound(desired_risk, delta, kappa, residuals, split= True)
    risk_dict_max_prob[str(desired_risk)] = [theta, b_star]


0.02 & 0.0177 & 0.3723 & 0.0200 & 0.3773 & 0.0200  \\
0.10 & 0.0964 & 0.6796 & 0.0957 & 0.6778 & 0.1000  \\
0.15 & 0.1460 & 0.8077 & 0.1471 & 0.8086 & 0.1500  \\
0.20 & 0.1958 & 0.9147 & 0.1945 & 0.9138 & 0.2000  \\
0.25 & 0.2442 & 1.0000 & 0.2445 & 1.0000 & 0.2485  \\
0.30 & 0.2429 & 1.0000 & 0.2457 & 1.0000 & 0.2472  \\
1.00 & 0.2442 & 1.0000 & 0.2444 & 1.0000 & 0.2485  \\


In [5]:
risk_dict_max_prob

{'0.02': [0.98898286, 0.019981025674879048],
 '0.1': [0.70664036, 0.09998997251228992],
 '0.15': [0.5008844, 0.14999056480335166],
 '0.2': [0.2773202, 0.19999532695157785],
 '0.25': [0.022015542, 0.24850052352575064],
 '0.3': [0.022936972, 0.24722394139462103],
 '1.0': [0.022015542, 0.24854849946431418]}

#### Method 2: **Vector Scaling**

In [6]:
import sys
sys.path.append("/teamspace/studios/this_studio/Selective_Prediction_VQA")
from calibration_methods import calibrator as cal

In [7]:
def load_calibrator(path, calibrator_type = "vector_calibrator", device = 'cpu'):
    dict = torch.load(path)
    if calibrator_type == "vector_calibrator":
        cali = cal.VectorScaling(bias=dict['biasFlag'], 
                                 weights= dict['weights'],
                                 num_label = dict['num_label'],
                                 device=device,
                                 print_verbose= False)
        cali.temperature = dict['temperature']
        cali.bias = dict['bias']
    else:
        #todo
        return None
    return cali

In [8]:
path = "/teamspace/studios/this_studio/Selective_Prediction_VQA/calibration_methods/scaling/vector_calibrator.pt"
cali = load_calibrator(path, calibrator_type = "vector_calibrator", device='cuda')

In [9]:
data_path = "/teamspace/studios/this_studio/Selective_Prediction_VQA/predictions/logits_and_labels/"
NUM_BATCH = 2139
residuals = []
kappa = []
softmax = torch.nn.Softmax(dim=1)
for batch_no in range(NUM_BATCH):
    file_name = "Logits_and_labels" + str(batch_no) + ".pt"
    data = torch.load(data_path + file_name)
    
    for logits, labels in zip(data['logits'], data['labels']):
        if(len(labels)) == 0:
            continue
        cali.calibrate(logits)
        logits = torch.from_numpy(logits)
        
        prob = softmax(logits)
        # print(prob)
        # print(torch.max(prob).numpy())
        # prob.to('cpu')
        kappa.append(torch.max(prob).numpy())
        idx = torch.argmax(prob)
        # print(prob[0][idx])
        # print(labels)
        residuals.append(idx.item() != max_occurence(labels))
        # print(kappa)
        # print(residuals)
        # break
    # break
        
    
# kappa, residuals = sort(kappa, residuals )

In [10]:
kappa = np.array(kappa)
residuals = np.array(residuals)
risk_dict_vec_scal = {}
risk_stars = [0.02, 0.1, 0.15, 0.20, 0.25,0.30, 1.0] # desired risk
delta = 0.1 ## confidence
for desired_risk in risk_stars:
    bound_cal = risk_control()
    [theta, b_star] = bound_cal.bound(desired_risk, delta, kappa, residuals, split= True)
    risk_dict_vec_scal[str(desired_risk)] = [theta, b_star]


0.02 & 0.0182 & 0.3759 & 0.0193 & 0.3723 & 0.0200  \\
0.10 & 0.0972 & 0.6842 & 0.0991 & 0.6852 & 0.1000  \\
0.15 & 0.1469 & 0.8090 & 0.1458 & 0.8064 & 0.1500  \\
0.20 & 0.1967 & 0.9190 & 0.1979 & 0.9178 & 0.2000  \\
0.25 & 0.2426 & 1.0000 & 0.2460 & 1.0000 & 0.2460  \\
0.30 & 0.2427 & 1.0000 & 0.2459 & 1.0000 & 0.2460  \\
1.00 & 0.2436 & 1.0000 & 0.2450 & 1.0000 & 0.2470  \\


In [11]:
risk_dict_vec_scal

{'0.02': [0.9891044, 0.019991926736553832],
 '0.1': [0.6969989, 0.09999231656799441],
 '0.15': [0.50149685, 0.14999694757617216],
 '0.2': [0.26786566, 0.19999246925641007],
 '0.25': [0.022015542, 0.24597829345634653],
 '0.3': [0.022936972, 0.2460358094776587],
 '1.0': [0.022310337, 0.2469848012272543]}

In [12]:

risk_dict = {
    'max_prob' : risk_dict_max_prob,
    'vector_scaling_calibration' : risk_dict_vec_scal
}
path = "/teamspace/studios/this_studio/Selective_Prediction_VQA/risk_bounds/"
torch.save(risk_dict, path + "risk_bound.pt")


In [ ]:
np.sum(residuals)

51211

In [ ]:
residuals.size

209608

In [ ]:
np.sum(residuals)/residuals.size

0.24431796496316935